In [1]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import scipy

from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt

import psycopg2
import psycopg2.extras
from pymongo import MongoClient

import gc

import os

import multiprocessing as mp
from multiprocessing import shared_memory

import time

from IPython.display import Image, HTML
pd.set_option('display.max_colwidth', -1)

%load_ext autoreload

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_9368\3276713142.py:24: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
%autoreload 2
from utility import utils
from Recommender import A_1, A_3
from utility.Network import ConnectModule
from engine import profiling_engine
from Recommender.A_1 import A_1ALgorithm
from Recommender.A_3 import A_3ALgorithm
from Preprocessing import common
from Preprocessing import common,A_1_preprocessing,A_3_preprocessing

In [3]:
client = MongoClient(host='10.10.224.28',
                     port=27017,
                     username='vida',
                     password='qwer123$')

db_datamart = client['datamart']

In [4]:
class RecommenderSystemA1(ConnectModule):
    def __init__(self, host_ip, DBname, port, username, password, email_id, email_pw, from_date, to_date, client):
        super().__init__(host_ip, DBname, port, username, password, email_id, to_date, client)
        self.from_date = from_date
        self.to_date = to_date

    def load_table(self):
        self.NOSQL = {'dm_orderseries_recommend': {'SQL': {}}}
        super().load_table(connect_close=False)
        productid = tuple(self.dataframe_dm_orderseries_recommend.loc[
                              self.dataframe_dm_orderseries_recommend['productId'].apply(
                                  lambda x: len(str(x).split('_')) <= 1),
                              'productId'].astype(int).unique())
        self.brand_favorite = {"SQL": "select tfb.\"brand\"" + (
            ", tfb.\"buyer\"") + (
                                          ", tfb.\"created_at\" at time zone 'Asia/Seoul'") + (
                                          " from tb_favorite_brand tfb"),
                               "columns": ['brandId', 'buyerId', 'yeramon']}
        self.wishes = {"SQL": "select wis.\"brandId\"" + (
            ", \"userId\"") + (
                                  ", \"productId\"") + (
                                  ", \"createdAt\"") + (
                                  " from wishes wis where ") + (
                                  "wis.status = 100 "),
                       "columns": ['brandId', 'buyerId', 'productId', 'createdAt']}
        self.products = {"SQL": "select id" + (
            ", \"name\"->'ko'") + (
                                    ", \"appImages\"") + (
                                    ", \"data\"") + (
                                    ", \"brand\"") + (
                                    ", \"status\"") + (
                                    ", \"createdAt\" at time zone 'Asia/Seoul'") + (
                                    " from products where ") + (
                                    f"id in {productid}"),
                         "columns": ['productId', 'name', 'images', 'SKUdata', 'brand', 'status', 'createdAt']}

        self.blacklist = {'SQL': "select * from tb_exception_intro_products",
                          'columns': ['id', 'productId', 'delete_at', 'delete_id']}
        self.NOSQL = {'dm_buyer': {'SQL': {}}}
        super().load_table()

    def run(self):
        if not hasattr(self, 'dataframe_dm_buyer'):
            self.load_table()

        self.dataframe_products = \
            self.dataframe_products[
                ~self.dataframe_products['productId'].isin(self.dataframe_blacklist['productId'])].reset_index(
                drop=True)
        # 상품 정보 전처리
        merchandise_reduced, self.orderseries_merchandise = \
                                common.merchandise_preprocessing(self.dataframe_products,
                                                                 self.dataframe_dm_orderseries_recommend)
        # 단골브랜드 전처리
        favorite_pivot = common.favorite_preprocessing(self.dataframe_brand_favorite,
                                                       self.orderseries_merchandise)
        # 위시리스트 전처리
        wishes_reduced, buyerId_dict, buyerId_favorite_dict, productId_dict, productId_set = \
            common.wishes_preprocessing(self.dataframe_wishes, favorite_pivot, self.orderseries_merchandise)
        # SVD Matrix Factorization + 코사인 유사도
        self.csm_total = common.svd_preprocessing(favorite_pivot)
        # 카테고리 데이터 전처리
        favorite_pivot_Id_set, product_profile_numpy, product_profile_Id = \
            A_1_preprocessing.product_profile_preprocessing(self.orderseries_merchandise, favorite_pivot)
        # 위시리스트 피보팅
        wishes_coo = A_1_preprocessing.wishes_preprocessing(buyerId_dict, productId_dict, wishes_reduced)
        # 단골브랜드 Top-N 유사도 도출(Profiling_engine)
        result, param = profiling_engine(self.csm_total,
                                         [i for i in range(0, self.csm_total.shape[0])],
                                         k=20,
                                         threshold=0.2,
                                         core=8)
        # 타겟 상품 전처리
        target_product_list = A_1_preprocessing.target_product_list_preprocessing(self.orderseries_merchandise,
                                                                                  favorite_pivot, result)
        # 상품 전처리
        product_list = A_1_preprocessing.product_list_preprocessing(favorite_pivot, self.orderseries_merchandise,
                                                                    result)
        # A-1 알고리즘부
        self.retrieval_dict, self.params_dict = A_1ALgorithm(target_product_list,
                                                             product_list,
                                                             product_profile_Id,
                                                             product_profile_numpy,
                                                             favorite_pivot_Id_set,
                                                             productId_set,
                                                             productId_dict,
                                                             wishes_coo,
                                                             drop_duplicated=True)

In [5]:
test = RecommenderSystemA1('restore.cy1gcw3mvrbf.ap-northeast-2.rds.amazonaws.com',
              'linkshops',
              '5432',
              'linkshops',
              '$dpdlvmflf4',
              'ji.kwon@linkshops.com','cjswp25*',
              '','',
                client)

In [6]:
start_time = time.time()
test.run()
print(time.time() - start_time)

NoSQLTable : dm_orderseries_recommend Load Complete
Table : products Load Complete
Table : wishes Load Complete
Table : blacklist Load Complete
Table : brand_favorite Load Complete
NoSQLTable : dm_buyer Load Complete
1613
(2730,)
0
0.04898691177368164
100
4.516253232955933
200
8.716477155685425
300
13.069498062133789
400
17.400954484939575
500
21.68428087234497
600
25.76830267906189
700
30.10811996459961
800
34.38277530670166
900
38.797799825668335
1000
42.932610273361206
1100
47.101853370666504
1200
51.182679891586304
1300
55.60428333282471
1400
59.80592322349548
1500
64.13261866569519
1600
68.29029536247253
1700
72.55874872207642
1800
76.71619415283203
1900
81.04314517974854
2000
85.2458336353302
2100
89.45504117012024
2200
93.62214517593384
2300
97.8812952041626
2400
101.96887803077698
2500
106.21250653266907
2600
110.39628291130066
2700
114.6820764541626
2800
118.9306116104126
2900
123.05456566810608
3000
127.24377036094666
3100
131.45239067077637
3200
135.71190071105957
3300
139.9

2570
1256.5340056419373
2580
1258.3348126411438
2590
1260.07732629776
2600
1265.4398665428162
2610
1267.3672609329224
2620
1268.6640086174011
2630
1271.4966208934784
2640
1274.2187719345093
2650
1281.3794293403625
2660
1285.371473312378
2670
1286.7133958339691
2680
1287.8980793952942
2690
1289.5387518405914
2700
1292.3477265834808
2710
1307.7181055545807
2720
1311.5217504501343
2730
1314.990581035614
2740
1317.1704931259155
2750
1321.0348303318024
2760
1324.400197505951
2770
1325.9308207035065
2780
1328.3638038635254
2790
1330.6525490283966
2800
1333.7635102272034
2810
1335.3713426589966
2820
1337.4732203483582
2830
1340.1615307331085
2840
1349.4831504821777
2850
1351.0214977264404
2860
1353.1710822582245
2870
1357.1638238430023
2880
1361.1253452301025
2890
1362.3612842559814
2900
1363.521868467331
2910
1364.767076253891
2920
1382.159993648529
2930
1383.3592443466187
2940
1387.371798992157
2950
1391.4723534584045
2960
1394.5839080810547
2970
1398.5439467430115
2980
1406.2251362800598
2

In [8]:
test.retrieval_dict[154936]

KeyError: 154936

In [ ]:
a = pd.concat([DataFrame(i).T for i in test.retrieval_dict.items()])
a = a.reset_index(drop = True)

b = pd.concat([DataFrame(i).T for i in test.params_dict.items()])
b = b.reset_index(drop = True)

recommened_brand_for_debug = pd.merge(a, b, on = 0)
recommened_brand_for_debug.columns = ['buyerId','retrieval','params']

recommened_brand_for_debug['retrieval'] = recommened_brand_for_debug['retrieval'].astype('str')
recommened_brand_for_debug['params'] = recommened_brand_for_debug['params'].astype('str')

In [76]:
recommened_brand_for_debug['type'] = 0

In [66]:
col_dm = db_datamart['recommend_brand_debug']
db_datamart.drop_collection('recommend_brand_debug')

{'ok': 0.0,
 'errmsg': 'ns not found',
 'code': 26,
 'codeName': 'NamespaceNotFound'}

In [74]:
col_dm.insert_many(recommened_brand_for_debug.T.to_dict().values())

In [65]:
client = MongoClient(host='10.10.224.28',
                     port=27017,
                     username='vida',
                     password='qwer123$')

db_datamart = client['datamart']

In [39]:
flattened_list = \
utils.afterprocessing_minimum_num(test.dataframe_products, 
                                  test.retrieval_dict, 
                                  test.params_dict, 
                                  test.orderseries_merchandise,
                                  30,
                                  0)

C:\Users\Administrator\Documents\Recommender System\utility\utils.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories['cats'] = categories['cat_0'] + categories['cat_1'].fillna('') + categories['cat_2'].fillna('')


In [40]:
flattened_list = flattened_list[['buyerId','productId','parameter','type']]

In [ ]:
col_dm.insert_many(flattened_list.T.to_dict().values())

In [64]:

conn = psycopg2.connect("host=restore.cy1gcw3mvrbf.ap-northeast-2.rds.amazonaws.com port=5432 user=linkshops password=$dpdlvmflf4 dbname=linkshops")

curs = conn.cursor()



In [13]:
tuples = [tuple(x) for x in flattened_list.to_numpy()]

In [20]:
tuples[0]

(1226,
 [7116695,
  7131141,
  7130025,
  7116695,
  7165712,
  7060208,
  6892022,
  7005303,
  7056917,
  7056761,
  7027087,
  7069224,
  7006062,
  6668282,
  7044689,
  6920027,
  7130025,
  7026261,
  7137249,
  7204799,
  7095052,
  6527747,
  7084653,
  7166482,
  6680501,
  7165712,
  7027087,
  7116377,
  7132549,
  6998065,
  7024293,
  6976584,
  7044440,
  6749492,
  7034019,
  7055356,
  6892022,
  7145361,
  6977483,
  6924541,
  7137249,
  6539502,
  6998041,
  6892022,
  6879033,
  6527747,
  6527747,
  5512001,
  6465150,
  7204799,
  7041578,
  6998065,
  6969781,
  6405889,
  7184760,
  6979320,
  7056761,
  7075486,
  7120809,
  6527747,
  7130025,
  7165712,
  7131141,
  7084653,
  7099628,
  7044440,
  7075433,
  7026261,
  6969779,
  6998041,
  7013350,
  6860414,
  6677173,
  6733983,
  6475728,
  7092274,
  7116377,
  6527747,
  7027087,
  7137249,
  7044689,
  6337631,
  6879033,
  6879033,
  7054171,
  7055539,
  6527747,
  7039748,
  6527747,
  7132549,
  7

In [60]:
curs.execute('DELETE FROM public.recommended_brand WHERE type::integer = 0')

In [65]:
psycopg2.extras.execute_values(curs,
                              'DELETE FROM public.recommended_brand WHERE type::integer = %s',
                               [[0]],
                               template=None)
conn.commit()

In [66]:
psycopg2.extras.execute_values(curs, 
                               'INSERT INTO public.recommended_brand VALUES %s',
                              tuples,
                              template=None,
                              page_size=1000)
conn.commit()

In [68]:
start_time = time.time()
col_dm = db_datamart['recommend_brand']
db_datamart.drop_collection('recommend_brand')

col_dm.insert_many(flattened_list.T.to_dict().values())
        
print(time.time() - start_time)

3.163964033126831


In [187]:
set([int(item) for items in test.retrieval_dict[159775].values() for item in items]).difference(
    flattened_list[flattened_list['buyerId'] == 159775]['productId'].iloc[0])

set()

In [188]:
set(flattened_list[flattened_list['buyerId'] == 159775]['productId'].iloc[0]).difference(
    [int(item) for items in test.retrieval_dict[159775].values() for item in items])

{3034210,
 4771886,
 6211489,
 6273750,
 6289446,
 6292039,
 6362071,
 6365967,
 6434672,
 6580045,
 6583527,
 6595174,
 6629036,
 6658793,
 6716197,
 6964930,
 7005119,
 7035964,
 7038984,
 7071318,
 7089584,
 7126828,
 7138876}

In [189]:
len(set(flattened_list[flattened_list['buyerId'] == 159775]['productId'].iloc[0]).difference(
    [int(item) for items in test.retrieval_dict[159775].values() for item in items]))

23

In [198]:
import sys

In [160]:
buyer_num = np.random.choice(list(test.retrieval_dict.keys()))
print(buyer_num)

121736


In [161]:
keys = list(test.retrieval_dict[buyer_num].keys())
values = flattened_list.loc[flattened_list['buyerId'] == buyer_num, 'productId'].iloc[0]
params = flattened_list.loc[flattened_list['buyerId'] == buyer_num, 'parameter'].iloc[0]

In [162]:
temp_total = utils.making_image_list_flattened(keys, values, params, test.orderseries_merchandise)

C:\Users\Administrator\Documents\Recommender System\utility\utils.py:90: FutureWarning: In a future version, passing float-dtype values containing NaN and an integer dtype will raise IntCastingNaNError (subclass of ValueError) instead of silently ignoring the passed dtype. To retain the old behavior, call Series(arr) or DataFrame(arr) without passing a dtype.
  Series(temp_total['target_prod'].unique(), name='productId', dtype='int'),


In [163]:
HTML(temp_total.to_html(escape = False))

In [111]:
keys = test.retrieval_dict[buyer_num].keys()
values = test.retrieval_dict[buyer_num].values()
params = test.params_dict[buyer_num].values()

In [112]:
size = (200,150)

In [114]:
HTML(temp_total.to_html(escape = False))

In [ ]:
retrieval_dict = test.retrieval_dict

evaluation_dict = dict()
cold_start_dict = dict()

for num,i in enumerate(list(retrieval_dict.keys())):
    unlist_list = list()
    # 추천 결과 X
    if isinstance(retrieval_dict[list(retrieval_dict.keys())[num]], list):
        evaluation_dict.update({i : {'precision' : np.nan,
                                     'recall' : np.nan}})
        if retrieval_dict[list(retrieval_dict.keys())[num]][0] == 'no_prod':
            cold_start_dict.update({i : {'no_prod' : 1}})
        elif retrieval_dict[list(retrieval_dict.keys())[num]][0] == 'no_target':
            cold_start_dict.update({i : {'no_target' : 1}})
        elif retrieval_dict[list(retrieval_dict.keys())[num]][0] == 'no_wishlist':
            cold_start_dict.update({i : {'no_wishlist' : 1}})
        continue
    [unlist_list.extend(i) for i in retrieval_dict[list(retrieval_dict.keys())[num]].values()]
    if len(unlist_list) == 0:
        evaluation_dict.update({i : {'precision' : np.nan,
                             'recall' : np.nan}})
        cold_start_dict.update({i : {'no_recommend' : 1}})
        continue
    tp = set(retrieval_dict[list(retrieval_dict.keys())[num]].keys()).intersection(unlist_list)
    
    evaluation_dict.update({i : {'precision' : len(tp) / len(unlist_list),
                                 'recall' : len(tp) / len(retrieval_dict[list(retrieval_dict.keys())[num]].keys())}})

In [538]:
def making_image_list(key, values, params,
                      size = (200,150)):

    temp_total = DataFrame()
    for i,j,k in zip(keys, values, params):
        k = np.array(k)
        temp = DataFrame({'productId' : j,
                          'params' : k[np.isnan(k) == False]})
        temp['target_prod'] = i
        if len(j) == 0:
            temp = DataFrame({'target_prod' : i,
                              'productId' : [np.nan],
                              'params' : [np.nan]})
        temp_total = pd.concat([temp_total, temp])
    urls = pd.merge(orderseries_merchandise,
                    Series(temp_total['target_prod'].unique(), name = 'productId', dtype='str')
                    )[['productId','images',]].drop_duplicates().values

    [os.system("curl http:" + i + f" > ./temp/target/{num}.jpg") for num, i in urls]
    frame_image = pd.concat([
                    DataFrame({'target_prod' : num,
                            'target_img' : [f'<img src=./temp/target/{num}.jpg + \
                             style="width:{size[0]}px;height:{size[1]}px;">']}) for num, i in urls])    
    frame_image = frame_image.astype({'target_prod' : str})
    temp_total = pd.merge(temp_total,frame_image)
    urls = pd.merge(orderseries_merchandise,
            Series(temp_total['productId'].dropna().unique().astype('int'), name = 'productId', dtype='str')
            )[['productId','images','status']].drop_duplicates().values

    [os.system("curl http:" + i + f" > ./temp/retrieval/{num}.jpg") for num, i, status in urls]
    frame_image = pd.concat([
                    DataFrame({'productId' : num,
                            'product_img' : [f'<img src=./temp/retrieval/{num}.jpg + \
                             style="width:{size[0]}px;height:{size[1]}px;">'],
                             'status' : status}) for num, i, status in urls])    
    frame_image['productId'] = frame_image['productId'].astype('str')
    temp_total = pd.merge(temp_total,
             frame_image,
             how = 'left')
    return temp_total

In [496]:
def afterprocessing_minimum_num(dataframe_products, retrieval_dict, params_dict, minimum_num):
    categories = dataframe_products[['productId','cat_0','cat_1','cat_2', 'status']]
    categories['cats'] = categories['cat_0'] + categories['cat_1'].fillna('') + categories['cat_2'].fillna('')

    flattened_list = utils.flatten_result(retrieval_dict, params_dict, 0)
    equal_zero_ind = flattened_list['productId'].apply(lambda x : len(x) == 0) 
    flattened_list = flattened_list[~equal_zero_ind].reset_index(drop = True)

    except_ind = flattened_list['productId'].apply(lambda x : isinstance(x[0], str))
    flattened_list = flattened_list[~except_ind].reset_index(drop = True)

    flattened_list['count'] = flattened_list['productId'].apply(lambda x : minimum_num - len(x))
    filling_needed = flattened_list[flattened_list['count'] > 0][['buyerId','count']]

    weights = orderseries_merchandise.groupby('productId')['finalQuantity'].sum().reset_index()
    weights['productId'] = weights['productId'].astype('int')
    categories = pd.merge(categories, weights, on = 'productId', how = 'left')

    afterprocessing_list = list()
    for _, (buyerId, count) in filling_needed.iterrows():
        cat = [categories[categories['productId'] == i]['cats'].values[0] for i in list(retrieval_dict[buyerId].keys())]
        param = Series(cat).value_counts()
        param = softmax_function(param)
        portion = [(i[0], math.floor(count * i[1])) 
                     if math.floor(count * i[1])> 0 
                     else (i[0], 1) for i in param.items()]
        break_point = np.cumsum(np.array(portion)[:,1].astype('int')) <= count
        if break_point.shape[0] > 1:
            portion = portion[0:np.max(np.where(break_point)) + 1]

        if np.sum(np.array(portion)[:,1].astype('int')) < count:
            remain_num = count - np.sum(np.array(portion)[:,1].astype('int'))
            n = 1
            temp = [1 for i in range(len(portion))]
            while True:
                if remain_num > len(temp):
                    portion = [(i[0], i[1] + j) for i,j in zip(portion, temp)]
                    remain_num -= len(temp)
                else:
                    temp = temp[0:remain_num] + [0 for i in range(0, len(temp) - remain_num)]
                    portion = [(i[0], i[1] + j) for i,j in zip(portion, temp)]
                    break
        random_sampled = list()
        for i in portion:
            prod_temp = categories[categories['cats'] == i[0]]['productId']
            weight_numerator = categories[categories['cats'] == i[0]]['finalQuantity'].to_numpy() + 1
            weight_denominator = np.sum(categories[categories['cats'] == i[0]]['finalQuantity'].to_numpy() + 1)
            sampling_weight = weight_numerator / weight_denominator 
            random_sampled.append(np.random.choice(prod_temp, 
                                                   size = i[1],
                                                   p = sampling_weight,
                                                   replace = False))
        afterprocessing_list.append(np.concatenate(random_sampled))

    for (_ , (buyerId, _)), j in zip(filling_needed.iterrows(), afterprocessing_list):
        unpack = flattened_list.loc[flattened_list['buyerId'] == buyerId, 'productId']
        concat = np.concatenate([unpack.values[0], j])
        flattened_list.at[int(np.where(flattened_list['buyerId'] == buyerId)[0]), 'productId'] = concat.tolist()
    flattened_list = flattened_list[['buyerId','productId','parameter','type']]
    
    return flattened_list

In [69]:
class RecommenderSystemA3(ConnectModule):
    def __init__(self, host_ip, DBname, port, username, password, email_id, email_pw, from_date, to_date, client):
        super().__init__(host_ip, DBname, port, username, password, email_id, to_date, client)
        self.from_date = from_date
        self.to_date = to_date

    def load_table(self):
        self.NOSQL = {'dm_orderseries_recommend': {'SQL': {}}}
        super().load_table(connect_close=False)
        productid = tuple(self.dataframe_dm_orderseries_recommend.loc[
                              self.dataframe_dm_orderseries_recommend['productId'].apply(
                                  lambda x: len(str(x).split('_')) <= 1),
                              'productId'].astype(int).unique())
        self.brand_favorite = {"SQL": "select tfb.\"brand\"" + (
            ", tfb.\"buyer\"") + (
                                          ", tfb.\"created_at\" at time zone 'Asia/Seoul'") + (
                                          " from tb_favorite_brand tfb"),
                               "columns": ['brandId', 'buyerId', 'yeramon']}
        self.wishes = {"SQL": "select wis.\"brandId\"" + (
            ", \"userId\"") + (
                                  ", \"productId\"") + (
                                  ", \"createdAt\"") + (
                                  " from wishes wis where ") + (
                                  "wis.status = 100 "),
                       "columns": ['brandId', 'buyerId', 'productId', 'createdAt']}
        self.products = {"SQL": "select id" + (
            ", \"name\"->'ko'") + (
                                    ", \"appImages\"") + (
                                    ", \"data\"") + (
                                    ", \"brand\"") + (
                                    ", \"status\"") + (
                                    ", \"createdAt\" at time zone 'Asia/Seoul'") + (
                                    " from products where ") + (
                                    f"id in {productid}"),
                         "columns": ['productId', 'name', 'images', 'SKUdata', 'brand', 'status', 'createdAt']}

        self.blacklist = {'SQL': "select * from tb_exception_intro_products",
                          'columns': ['id', 'productId', 'delete_at', 'delete_id']}
        self.NOSQL = {'dm_buyer': {'SQL': {}}}
        super().load_table()

    def run(self):
        if not hasattr(self, 'dataframe_dm_buyer'):
            self.load_table()
        self.dataframe_products = \
            self.dataframe_products[
                ~self.dataframe_products['productId'].isin(self.dataframe_blacklist['productId'])].reset_index(
                drop=True)
        # 상품 정보 전처리
        merchandise_reduced, self.orderseries_merchandise = \
                                common.merchandise_preprocessing(self.dataframe_products,
                                                                 self.dataframe_dm_orderseries_recommend)
        # 단골 브랜드 전처리
        favorite_pivot = common.favorite_preprocessing(self.dataframe_brand_favorite,
                                                       self.orderseries_merchandise)
        # 위시리스트 전처리
        wishes_reduced, buyerId_dict, buyerId_favorite_dict, productId_dict, productId_set = \
            common.wishes_preprocessing(self.dataframe_wishes, favorite_pivot, self.orderseries_merchandise)
        # SVD Matrix Factorization + 코사인 유사도
        self.csm_total = common.svd_preprocessing(favorite_pivot)
        # 위시리스트 피보팅
        wishes_coo = A_3_preprocessing.wishes_preprocessing(buyerId_dict, productId_dict, wishes_reduced)
        # 단골브랜드 Top-N 유사도 도출
        result, param = profiling_engine(self.csm_total,
                                         [i for i in range(0, self.csm_total.shape[0])],
                                         k=20,
                                         threshold=0.2)
        # Top - N 유사도 바이어 인덱스 -> ID 변환
        result_buyer = [favorite_pivot.iloc[i[np.isnan(i) == False].astype('int'), 0].values for i in result]
        IdBuyer_favorite_dict = {j: i for i, j in buyerId_favorite_dict.items()}
        merchandise_reduced['cats'] = merchandise_reduced['cat_0'] + merchandise_reduced['cat_1'] + merchandise_reduced[
            'cat_2']
        # 카테고리 전처리
        merchandise_reduced = merchandise_reduced[~merchandise_reduced['cats'].isna()]
        wishes_reduced = pd.merge(wishes_reduced,
                                  merchandise_reduced[['productId', 'cats']].astype({'productId': int})
                                  )
        wishes_reduced_dict = wishes_reduced.groupby('buyerId').apply(lambda x: x.values[:, 1:].tolist())
        # A-3 알고리즘부
        self.retrieval_dict, self.params_dict = A_3ALgorithm(favorite_pivot,
                                                             result_buyer,
                                                             wishes_coo,
                                                             wishes_reduced_dict,
                                                             productId_dict,
                                                             IdBuyer_favorite_dict,
                                                             drop_duplicated=True
                                                             )
                

In [70]:
test = RecommenderSystemA3('restore.cy1gcw3mvrbf.ap-northeast-2.rds.amazonaws.com',
              'linkshops',
              '5432',
              'linkshops',
              '$dpdlvmflf4',
              'ji.kwon@linkshops.com','cjswp25*',
              '','',
                client)

In [71]:
start_time = time.time()
test.run()
print(time.time() - start_time)

NoSQLTable : dm_orderseries_recommend Load Complete
Table : products Load Complete
Table : brand_favorite Load Complete
Table : wishes Load Complete
Table : blacklist Load Complete
NoSQLTable : dm_buyer Load Complete
1714
(2723,)
0
3.443276882171631
10
21.594709157943726
20
37.149086236953735
30
95.6050283908844
40
106.51706147193909
50
141.9271228313446
60
155.42269015312195
70
198.15852189064026
90
264.69121289253235
100
282.4686722755432
110
356.3939528465271
120
374.58706974983215
130
414.11830258369446
140
433.6998679637909
150
448.36728858947754
160
474.06185960769653
180
521.7019593715668
190
536.9030663967133
200
541.1797456741333
210
555.5760445594788
220
596.4785296916962
230
612.2691659927368
240
651.4288949966431
250
726.6582713127136
260
768.8537385463715
270
793.0291261672974
280
830.6243481636047
290
847.3991973400116
310
880.0344874858856
320
921.6148736476898
330
931.7210917472839
340
968.8850827217102
350
981.1651031970978
360
1011.5320546627045
370
1071.2097253799438

3640
10841.128618240356
3650
10888.407494783401
3660
10892.858558416367
3670
10921.001060247421
3680
10930.916462421417
3690
10952.366915225983


KeyboardInterrupt: 

In [9]:
retrieval_dict = test.retrieval_dict

evaluation_dict = dict()
cold_start_dict = dict()

for num,i in enumerate(list(retrieval_dict.keys())):
    unlist_list = list()
    # 추천 결과 X
    if isinstance(retrieval_dict[list(retrieval_dict.keys())[num]], list):
        evaluation_dict.update({i : {'precision' : np.nan,
                                     'recall' : np.nan}})
        if retrieval_dict[list(retrieval_dict.keys())[num]][0] == 'no_prod':
            cold_start_dict.update({i : {'no_prod' : 1}})
        elif retrieval_dict[list(retrieval_dict.keys())[num]][0] == 'no_target':
            cold_start_dict.update({i : {'no_target' : 1}})
        elif retrieval_dict[list(retrieval_dict.keys())[num]][0] == 'no_wishlist':
            cold_start_dict.update({i : {'no_wishlist' : 1}})
        continue
    [unlist_list.extend(i) for i in retrieval_dict[list(retrieval_dict.keys())[num]].values()]
    if len(unlist_list) == 0:
        evaluation_dict.update({i : {'precision' : np.nan,
                             'recall' : np.nan}})
        cold_start_dict.update({i : {'no_recommend' : 1}})
        continue
    tp = set(retrieval_dict[list(retrieval_dict.keys())[num]].keys()).intersection(unlist_list)
    
    evaluation_dict.update({i : {'precision' : len(tp) / len(unlist_list),
                                 'recall' : len(tp) / len(retrieval_dict[list(retrieval_dict.keys())[num]].keys())}})

In [10]:
DataFrame(evaluation_dict).T['precision'].mean()

0.0

In [11]:
DataFrame(evaluation_dict).T['recall'].mean()

0.0

In [12]:
a = DataFrame(evaluation_dict).T

print(a['recall'].isna().value_counts())
print(a['precision'].isna().value_counts())
print(((a['precision'].isna()) | (a['recall'].isna())).value_counts())
print(((a['precision'].isna()) & (a['recall'].isna())).value_counts())

False    4665
True     238 
Name: recall, dtype: int64
False    4665
True     238 
Name: precision, dtype: int64
False    4665
True     238 
dtype: int64
False    4665
True     238 
dtype: int64


In [84]:
test_2 = RecommenderSystemA3('restore.cy1gcw3mvrbf.ap-northeast-2.rds.amazonaws.com',
              'linkshops',
              '5432',
              'linkshops',
              '$dpdlvmflf4',
              'ji.kwon@linkshops.com','cjswp25*',
              '','',
                client)

In [86]:
test_2.load_table()

NoSQLTable : dm_orderseries_recommend Load Complete
Table : brand_favorite Load Complete
Table : products Load Complete
Table : wishes Load Complete
Table : blacklist Load Complete
NoSQLTable : dm_buyer Load Complete


In [87]:
test_2.run()

1711
(2718,)
0
2.505890369415283


KeyboardInterrupt: 

In [145]:
dataframe_products = test.dataframe_products.copy()
orderseries_merchandise = test.orderseries_merchandise.copy()

In [146]:
flattened_list = \
utils.afterprocessing_minimum_num(dataframe_products, 
                                  test.retrieval_dict, 
                                  test.params_dict, 
                                  orderseries_merchandise,
                                  30)

C:\Users\Administrator\Documents\Recommender System\utility\utils.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories['cats'] = categories['cat_0'] + categories['cat_1'].fillna('') + categories['cat_2'].fillna('')


In [163]:
buyer_num = np.random.choice(list(test.retrieval_dict.keys()))
print(buyer_num)

102482


In [166]:
keys = list(test.retrieval_dict[buyer_num].keys())
values = flattened_list.loc[flattened_list['buyerId'] == buyer_num, 'productId'].iloc[0]
params = flattened_list.loc[flattened_list['buyerId'] == buyer_num, 'parameter'].iloc[0]

In [167]:
temp_total = utils.making_image_list_flattened(keys, values, params, test.orderseries_merchandise)

In [168]:
HTML(temp_total.to_html(escape = False))

In [ ]:
buyer_num = np.random.choice(list(test.retrieval_dict.keys()))
print(buyer_num)

keys = list(test.retrieval_dict[buyer_num].keys())
values = flattened_list.loc[flattened_list['buyerId'] == buyer_num, 'productId'].iloc[0]
params = flattened_list.loc[flattened_list['buyerId'] == buyer_num, 'parameter'].iloc[0]

temp_total = utils.making_image_list_flattened(keys, values, params, test.orderseries_merchandise)

HTML(temp_total.to_html(escape = False))

In [159]:
set([int(item) for items in test.retrieval_dict[170403].values() for item in items]).difference(
    flattened_list[flattened_list['buyerId'] == 170403]['productId'].iloc[0])

set()

In [162]:
set(flattened_list[flattened_list['buyerId'] == 170403]['productId'].iloc[0]).difference(
    [int(item) for items in test.retrieval_dict[170403].values() for item in items])

{6304261,
 6404898,
 6549350,
 6558407,
 6839274,
 6941059,
 6952639,
 6965163,
 6976614,
 6982880,
 7033059,
 7062226,
 7082136,
 7083815,
 7100463,
 7169921,
 7171883,
 7177714,
 7211680,
 7217968,
 7224421,
 7225385,
 7247585,
 7250166}

In [110]:
HTML(temp_total.to_html(escape = False))

In [173]:
keys = test.retrieval_dict[buyer_num].keys()
values = test.retrieval_dict[buyer_num].values()
params = test.params_dict[buyer_num].values()

In [174]:
keys = np.array(list(keys), dtype = 'int')
values = np.array([np.array(i, dtype = 'int') for i in list(values)])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8476\4049117916.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([np.array(i, dtype = 'int') for i in list(values)])


In [186]:
temp_total = utils.making_image_list(keys, values, params, orderseries_merchandise, max_prod=100)

In [187]:
HTML(temp_total.to_html(escape = False))

In [196]:
orderseries_merchandise[orderseries_merchandise['productId'] == str(6436298)][['cat_0','cat_1','cat_2']].drop_duplicates()

cat_0         cat_1 cat_2
166938  WOMEN  DRESSES/SETS  Sets

In [197]:
orderseries_merchandise[orderseries_merchandise['productId'] == str(2450910)][['cat_0','cat_1','cat_2']].drop_duplicates()

cat_0         cat_1 cat_2
510178  WOMEN  DRESSES/SETS  Sets

In [25]:
HTML(temp_total.to_html(escape = False))